In [388]:
import pandas as pd

In [389]:

pip install tapi-yandex-metrika


Note: you may need to restart the kernel to use updated packages.


In [390]:
pip install simplejson

Note: you may need to restart the kernel to use updated packages.


In [391]:
from tapi_yandex_metrika import YandexMetrikaLogsapi

ACCESS_TOKEN = "AQAAAAAfqkFsAAe6kJvKxOViX0QAogM1yZunC9o"
COUNTER_ID = "73465***"

client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': COUNTER_ID},
    # Download the report when it will be created
    wait_report=True,
)
params={
    "fields": "ym:s:parsedParamsKey1,ym:s:parsedParamsKey2,ym:s:date,ym:s:clientID", #fields
    "source": "visits", #source
    #"dimensions":"",
    "date1": "2022-02-01",
    "date2": "2022-02-15",
}



result = client.create().post(params=params) #info
request_id = result["log_request"]["request_id"] #info

report = client.download(requestId=request_id).get()
data = report().data

#print(data)



#print("Raw data")
#data = report.data

#print("Column names")
#print(report.columns)

#Transform to values
#print(report().to_values())

# Transform to lines
#print(report().to_lines())

# Transform to dict
#print(report().to_dicts())

In [392]:
# не удалять
dicts = report().to_dicts()

In [393]:

df = pd.DataFrame(dicts)

In [394]:
df

,ym:s:parsedParamsKey1,ym:s:parsedParamsKey2,ym:s:date,ym:s:clientID
0,[],[],2022-02-07,1644258552254756397
1,[],[],2022-02-07,164421195376616812
2,[],[],2022-02-14,1644793454474444365
3,[],[],2022-02-03,1643850372803191058
4,"['phone_number','agent_id']","['+7 (963) 717-86-68','']",2022-02-02,1643785587896672368
...,...,...,...,...
9546,[],[],2022-02-04,1643947303400780664
9547,"['phone_number','agent_id','phone_number','ord...","['+7 (915) 084-97-71','','+7 (903) 258-49-18',...",2022-02-03,1641399171733900980
9548,[],[],2022-02-01,1643689636328050026
9549,[],[],2022-02-15,16449585611073056905


In [306]:
import requests

In [1003]:
# Адрес api метода для запроса get 
url_param = "https://api-metrika.yandex.net/stat/v1/data"
# Задаем параметры для API

mitrics_ym = ["ym:s:visits","ym:s:pageviews","ym:s:bounceRate","ym:s:pageDepth","ym:s:avgVisitDurationSeconds"]
dimensions_ym = ["ym:s:date"]

api_param = {
    "ids":"73465***", 
    "metrics": mitrics_ym,
    "dimensions": "ym:s:date",#,ym:s:<attribution>TrafficSource,ym:s:<attribution>SourceEngine,ym:s:gender",
    "date1":"2022-02-01",
    "date2":"2022-02-28",
    "sort":"ym:s:date",
    #"accuracy":"full",
    #"limit":5
    }

header_params = {
    'GET': '/management/v1/counters HTTP/1.1',
    'Host': 'api-metrika.yandex.net',
    'Authorization': 'OAuth AQAAAAAfqkFsAAe6kJvKxOViX0QAogM1yZunC9o',
    'Content-Type': 'application/x-yametrika+json'
    }


# Отправляем get request (запрос GET)
response = requests.get(
    url_param,
    params=api_param,
    headers=header_params
)
result = response.json()
#query = result['query']
#data = result['data']
#print(query)
#print('======================')
#print(data)




 ###### metrics ##################
 # функция для создания столбцов из метрик(показателей)

def metrics(metr):
    spisok = []
    for one in result['data']:
        for key,value in one.items():
            if key == 'metrics':
                spisok.append(value[metr])
            
    final_df = pd.DataFrame(spisok)
    final_df = final_df.rename(columns = {0:metr})
    return final_df


 # конкатенация столбцов показтелей в единый датафрейм
    
tab_metrics = pd.concat([metriks(0), metriks(1), metriks(2), metriks(3), metriks(4)], axis=1) # количество элементов
 # в конкатенации должно соответствовать количеству метрик(показателей)

tab_metrics.columns = mitrics_ym


 ###### dimensions ##################
 # функция для создания столбцов из дименшенсов(параметров)

def dimensions(dims):
    spisok = []
    for one in result['data']:
        for key,value in one.items():
            if key == 'dimensions':
                spisok.append(value[dims])
            
    final_df = pd.DataFrame(spisok)
    final_df = final_df.rename(columns = {0:dims})
    return final_df


tab_dimensions = pd.concat([dimensions(0)], axis=1) # количество элементов
 # в конкатенации должно соответствовать количеству дименшинсов(параметров)

tab_dimensions.columns = dimensions_ym

 # конкатенируем параметры и показатели
tab_final = pd.concat([tab_dimensions, tab_metrics], axis=1)



In [1004]:
tab_final

,ym:s:date,ym:s:visits,ym:s:pageviews,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds
0,2022-02-01,447.0,574.0,23.489933,1.284116,144.807606
1,2022-02-02,435.0,512.0,26.666667,1.177011,111.289655
2,2022-02-03,492.0,617.0,36.991870,1.254065,109.012195
3,2022-02-04,446.0,541.0,36.098655,1.213004,72.948430
4,2022-02-05,230.0,261.0,28.695652,1.134783,97.039130
5,2022-02-06,293.0,379.0,27.645051,1.293515,72.187713
6,2022-02-07,687.0,899.0,38.136827,1.308588,70.502183
7,2022-02-08,741.0,962.0,32.118758,1.298246,79.546559
8,2022-02-09,560.0,821.0,22.142857,1.466071,123.403571
9,2022-02-10,746.0,1084.0,24.664879,1.453083,127.958445
